In [1]:
import functools
import sys
import math
#import scipy.io
sys.path.append("..")
import matplotlib.pyplot as plt
import numpy as np

import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()
import tensorflow_probability as tfp

import Test_Likelihood as tl
import ODE_Dynamics as od
import Positive_Symptom_fn as fn

from functools import partial

import scipy.stats as ss

import simulation as sim
import model as mdl

tfd = tfp.distributions
tfb = tfp.bijectors

n_sample = 20000
n_burnin = 10000
n_adaptation = 8000



In [11]:
# Simulate Data: N(150, 10^2)
# Parameters: mu, sigma
n_sample = 100
mu = 100
sigma = 1
data = sigma * np.random.randn(n_sample) + mu

mu0 = mu
sigma0 = sigma

def joint_log_prob(data, mu, sigma):
    return -np.sum(math.log(2*math.pi*(sigma**2))/2 + ((data-mu)**2)/(2 * (sigma**2)))

unnormalized_posterior_log_prob = partial(joint_log_prob, data)

joint_log_prob(data, mu, sigma)

-138.37338013732062

In [12]:
import functools
import sys
import math
    
sys.path.append("..")
import matplotlib.pyplot as plt
import numpy as np

import tensorflow.compat.v2 as tf
tf.enable_v2_behavior()
import tensorflow_probability as tfp

import Test_Likelihood as tl
import ODE_Dynamics as od
import Positive_Symptom_fn as fn

from functools import partial

import scipy.stats as ss

import simulation as sim
import model as mdl

tfd = tfp.distributions
tfb = tfp.bijectors

initial_state = [tf.convert_to_tensor(mu0, dtype = tf.float32),tf.convert_to_tensor(sigma0,dtype = tf.float32)]

unconstraining_bijectors = [
    tfb.Identity(),
    tfb.Exp()
]

def sample():
    return tfp.mcmc.sample_chain(
        num_results = n_sample,
        num_burnin_steps = n_burnin,
        current_state = initial_state,
        kernel = tfp.mcmc.SimpleStepSizeAdaptation(
            tfp.mcmc.TransformedTransitionKernel(
                inner_kernel = tfp.mcmc.HamiltonianMonteCarlo(
                    target_log_prob_fn = unnormalized_posterior_log_prob,
                    step_size = 0.1,
                    num_leapfrog_steps = 2),
                bijector = unconstraining_bijectors),
            num_adaptation_steps = n_adaptation),
        trace_fn=lambda _, pkr: pkr.inner_results.inner_results.is_accepted)

[mu, sigma], is_accepted = sample()
    

ValueError: Encountered `None` gradient.
  fn_arg_list: [<tf.Tensor: shape=(), dtype=float32, numpy=100.0>, <tf.Tensor: shape=(), dtype=float32, numpy=0.0>]
  grads: [None, <tf.Tensor: shape=(), dtype=float32, numpy=1.0>]